In [1]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability.python as tfp
from tensorflow_probability.python import math as tfp_math
tfd = tfp.distributions

# From nsc source code
from function import coupled_logarithm as coupled_log, coupled_exponential as coupled_exp

### **Gaussian function**

The Gaussian function is the form of:

$f(x) = a * exp(-\frac{(x - b)^2}{2c^2})$

For given real constants *a*, *b*, and non-zero *c*

### **1D Gaussian Distribution**

The formula of a 1D Gaussian distribution is the following:


$p(x ; \mu, \sigma) = \frac{1}{\sigma\sqrt{2\pi}}exp(-\frac{1}{2}\frac{(x - \mu)^2}{\sigma^2})$

Where
- *p* = probability density dunction of a normally distributed random variable *x*.
- *a* = $\frac{1}{\sigma\sqrt{2\pi}}$
- *b* = $\mu$
- *c* = $\sigma$

Sources: [Wikipedia](https://en.wikipedia.org/wiki/Gaussian_function), [Youtube](https://www.youtube.com/watch?v=eho8xH3E6mE&ab_channel=AlexanderIhler)

The tfp.distribution Normal function is written as the following:

```none
pdf(x; mu, sigma) = exp(-0.5 (x - mu)**2 / sigma**2) / Z
Z = (2 pi sigma**2)**0.5
```

Therefore:


```none
log_pdf(x; mu, sigma)

= log(exp(-0.5 (x - mu)**2 / sigma**2) / Z) 

= log(exp(-0.5 (x - mu)**2 / sigma**2)) - log(Z) 

= -0.5 (x - mu)**2 / sigma**2 - log(Z)

= -0.5 (x - mu)**2 / sigma**2 - log((2*pi*sigma**2)**0.5)

= -0.5 (x - mu)**2 / sigma**2 - log((2*pi*sigma**2)**0.5)

= -0.5 (x - mu)**2 / sigma**2 - 0.5*log(2*pi*sigma**2)

= -0.5 (x - mu)**2 / sigma**2 - 0.5*log(2*pi*sigma**2)

= -0.5 (x - mu)**2 / sigma**2 - 0.5*log(2*pi) + 0.5*log(sigma**2)

= -0.5 (x - mu)**2 / sigma**2 - 0.5*log(2*pi) + 0.5*2*log(sigma)

= -0.5 (x - mu)**2 / sigma**2 - 0.5*log(2*pi) + log(sigma)

```

Given:

```
log_unnormalized = -0.5 (x - mu)**2 / sigma**2
log_normalization = 0.5*log(2*pi) + log(sigma)
```

Then:

log_pdf(x; mu, sigma) = log_unnormalized - log_normalization

In [2]:
def _log_prob(self, x, loc, scale):
    scale = tf.convert_to_tensor(scale)
    log_unnormalized = -0.5 * tf.math.squared_difference(
        x / scale, loc / scale)
    log_normalization = tf.constant(
        0.5 * np.log(2. * np.pi), dtype=self.dtype) + tf.math.log(scale)
    return log_unnormalized - log_normalization

### **Multivariate Gaussian Distribution**

The formula of a multivariate Gaussian distribution is the following:


$p(x ; \mu, \Sigma) = \frac{1}{(2\pi)^{d/2}|\Sigma|^{1/2}}exp(-\frac{1}{2}(x - \mu)^T\Sigma^{-1}(x - \mu)^2)$

Where
- *p* = PDF is a multidimensional random variable *x*.
- *d* = # dimension
- $\Sigma$ = covariance matrix
- *T* = transpose

Sources: [Wikipedia](https://en.wikipedia.org/wiki/Gaussian_function), [Youtube](https://www.youtube.com/watch?v=eho8xH3E6mE&ab_channel=AlexanderIhler)

The tfp.distribution MultivariateNormalLinearOperator and TransformedDistribution function is written as the following:

```none
TO-DO
```

Where: TO-DO


### **1D Student-t Distribution**

The formula of a 1D Student-t distribution is the following:

$f(t) = \frac{\Gamma[(v + 1)/2]}{\sqrt{v\pi}\Gamma[v/2]}(1 + t^2/v)^{-(v+1)/2}$

Where:

$t = \frac{x - \mu}{\sigma}$

$v = DoF $

$\Gamma = Gamma function $

$d = 1D $

$ \sum = 1$

Sources: [Wikipedia](https://en.wikipedia.org/wiki/Multivariate_t-distribution)

The tfp.distribution Student-t function is written as the following:

The probability density function (pdf) is,
```none
pdf(x; df, mu, sigma) = (1 + y**2 / df)**(-0.5 (df + 1)) / Z
where,
y = (x - mu) / sigma
Z = abs(sigma) sqrt(df pi) Gamma(0.5 df) / Gamma(0.5 (df + 1))

or

pdf(x; df, mu, sigma) = (1 + y)**(-0.5 (df + 1)) / Z
where,
y = [(x - mu) / sigma]**2 / df
Z = abs(sigma) sqrt(df pi) Gamma(0.5 df) / Gamma(0.5 (df + 1))
```
 
Therefore:

```
log(pdf) = log((1 + y)**(-0.5 (df + 1)) / Z)
         = (-0.5*(df+1))*log(1+y) - log(Z)
         = (-0.5*(df+1))*log(1 + ((x-mu)/sigma)**2/df) - log(Z)
         = (-0.5*(df+1)) * log(((x-mu)/sigma)**2/df) - log(Z)
log(Z) = log(abs(sigma)*sqrt(df*pi)*Gamma(0.5*df) / Gamma(0.5*(df+1)))
log(Z) = log(abs(sigma)) + pi*log(df) + log(Gamma(0.5*df) - log(Gamma(0.5*(df+1))

```

Given:

```
log_unnormalized_prob = (-0.5*(df+1)) * log(((x-mu)/sigma)**2/df)
log_normalization = log(abs(sigma)) + pi*log(df) + log(Gamma(0.5*df) - log(Gamma(0.5*(df+1))
```

Then:

log_pdf(x; mu, sigma) = log_unnormalized_prob - log_normalization

In [3]:
def log_prob(x, df, loc, scale):
  # Writing `y` this way reduces XLA mem copies.
  y = (x - loc) * (tf.math.rsqrt(df) / scale)
  # y = (x - loc) / (tf.math.rsqrt(df) * scale) right instead ???
  log_unnormalized_prob = -0.5 * (df + 1.) * log1psquare(y)
  log_normalization = (
      tf.math.log(tf.abs(scale)) + 0.5 * tf.math.log(df) +
      0.5 * np.log(np.pi) + tfp_math.log_gamma_difference(0.5, 0.5 * df))
      # should this be pi * tf.math.log(df) instead?
  return log_unnormalized_prob - log_normalization

### **Multivariate Student-t Distribution**

The formula of a 1D Student-t distribution is the following:

$f(x; \mu,\Sigma) = \frac{\Gamma[(v + d)/2]}{\Gamma[v/2]v^{d/2}\pi^{d/2}|\Sigma|^{1/2}}(1 + \frac{1}{v}(x - \mu)^T\Sigma^{-1}(x - \mu))^{-(v+d)/2}$

Where:

$v = DoF$

$\Gamma = Gamma function$

$d = n Dimensions$

$\sum = covariance$

Sources: [Wikipedia](https://en.wikipedia.org/wiki/Multivariate_t-distribution)

The tfp.distribution MultivariateStudentTLinearOperator function is written as the following.

The probability density function (pdf) is:
```none
  pdf(x; df, loc, Sigma) = (1 + ||y||**2 / df)**(-0.5 (df + k)) / Z
  where:
  y = inv(Sigma) (x - loc)
  Z = abs(det(Sigma)) sqrt(df pi)**k Gamma(0.5 df) / Gamma(0.5 (df + k))
```

In formula form:

$df = DoF$

$k = nDimensions$

$y = Σ^{−1}(𝑥−𝜇)$

$z = \frac{|\Sigma|^{1/2}(df\pi)^{k/2}\Gamma[df/2]}{\Gamma[(df + k)/2]}$

Therefore:

```
log(pdf) = log((1 + ||y||**2 / df)**(-0.5 (df + k)) / Z)
         = (-0.5*(df+k))*log(1 + ||y||**2 / df) - log(Z)
         = (-0.5*(df+k))*log(1 + ((x-mu)/sigma)**2/df) - log(Z)
         = (-0.5*(df+k)) * log(((x-mu)/sigma)**2/df) - log(Z)
log(Z) = log(abs(sigma)*sqrt(df*pi)**k*Gamma(0.5*df) / Gamma(0.5*(df+k)))
log(Z) = log(abs(sigma)) + k*log_sqrt(df*pi) + log(Gamma(0.5*df)) - log(Gamma(0.5*(df+k))
log(Z) = log(abs(sigma)) + (k/2)*log(df*pi) + log(Gamma(0.5*df)) - log(Gamma(0.5*(df+k))
log(Z) = log(abs(sigma)) + (k/2)*(log(df)+log(pi)) + log(Gamma(0.5*df)) - log(Gamma(0.5*(df+k))

```

Given:

```
num_dims = k
mahalanobis = value = (x - mu)/sigma
_log_unnormalized_prob = (-(num_dims+df)/2) * log^2(mahalanobis/sqrt(df))
_log_normalization = log(Gamma(0.5*df)?? - log(Gamma(0.5*(df+num_dims))?? + (num_dims/2)*(log(df)+log(pi)) + log(abs(sigma))
```

Then:

log_pdf(x; mu, sigma) = log_unnormalized_prob - log_normalization

In [4]:
# What class MultivariateStudentTLinearOperator actually contains:

def _log_unnormalized_prob(self, value):
    df = tf.convert_to_tensor(self.df)
    value = value - self._loc
    value = self.scale.solve(value[..., tf.newaxis])

    num_dims = tf.cast(self.event_shape_tensor()[0], self.dtype)
    mahalanobis = tf.norm(value, axis=[-1, -2])
    return -(num_dims + df) / 2. * tfp_math.log1psquare(
        mahalanobis / tf.sqrt(df))

def _log_normalization(self):
    df = tf.convert_to_tensor(self.df)
    num_dims = tf.cast(self.event_shape_tensor()[0], self.dtype)
    return (tfp_math.log_gamma_difference(num_dims / 2., df / 2.) +
            num_dims / 2. * (tf.math.log(df) + np.log(np.pi)) +
            self.scale.log_abs_determinant())

def _log_prob(self, value):
    return self._log_unnormalized_prob(value) - self._log_normalization()

### **Multivariate Coupled Normal Distribution**

**PDF**

Let Z = NormMultiCoupled, d = n Dimensions, k = coupling value (kappa), dimMean = len($\mu$)

Z$(x ; \mu, \Sigma, \kappa, \alpha, len(\mu)) = \frac{|\Sigma|^{1/2}}{1 + (-1 + d)\kappa}$ if $\alpha$ = 1

Z$(x ; \mu, \Sigma, \kappa, \alpha, len(\mu)) = \frac{\sqrt{\pi}|\Sigma|^{1/2}\Gamma[\frac{1+(-1+d)\kappa}{2\kappa}]}{\sqrt{\kappa}\Gamma[\frac{1+d\kappa}{2\kappa}]}$ if $\alpha$ = 2

Compared to the Z from Multivariate Student-t:

Z = $\frac{|\Sigma|^{1/2}(df\pi)^{k/2}\Gamma[df/2]}{\Gamma[(df + k)/2]}$

Therefore, the Multivariate Coupled Normal PDF is the following:

$p(x ; \mu, \Sigma, \kappa, \alpha) = \frac{CoupledExpotential([(x−\mu)^𝑇|Σ|^{−1}(𝑥−\mu))]^{\alpha/2}, \kappa, len(\mu))^{-1/\alpha}}{Z(x ; \mu, \Sigma, \kappa, \alpha, len(\mu))}$

The probability density function (pdf) is:
```none

pdf(x; loc, Sigma, k, a, d) = coupled_exp(y^(a/2), k, len(loc))^(-1/a) / Z(x; loc, Simga, k, a, d)

where:
y = inv(Sigma)*(x - loc)
Z(x; loc, Simga, k, a, d) = [sqrt(pi) * abs(det(Sigma)) * Gamma((1+(-1+d)*k)/(2*k))] /                                                     [sqrt(k) * Gamma((1+dk)/(2k)]
 
_unnormalized_prob = coupled_exp(y^(a/2))^(-1/a)
_unnormalized_prob = coupled_exp((inv(scale)*(x - loc))^(a/2))^(-1/a)

_normalization = Z
_normalization = [sqrt(pi) * abs(det(scale)) * Gamma((1+(-1+d)*k)/(2*k))] /                                                    [sqrt(k) * Gamma((1+dk)/(2k)]
```

Therefore:

_prob(x; mu, sigma) = _unnormalized_prob / log_normalization

Given the existence of the following in the MultivariateCoupledNormal distribution class:

- self.loc (self._loc)
- self.scale
- self.kappa
- self.num_dim (equal to len(self.loc))

In [5]:
# HAVE NOT TESTED YET

def _unnormalized_prob(self, value):
    kappa = tf.convert_to_tensor(self.kappa)
    value = value - self._loc
    value = self.scale.solve(value[..., tf.newaxis])
    
    num_dims = tf.cast(self.event_shape_tensor()[0], self.dtype)
    # mahalanobis is equivalent to y
    mahalanobis = tf.norm(value, axis=[-1, -2])
    return coupled_exp(mahalanobis^(a/2), kappa, num_dim)^(-1/a)  

def _normalization(self):
    kappa = tf.convert_to_tensor(self.kappa)
    num_dims = tf.cast(self.event_shape_tensor()[0], self.dtype)
    # no gamma_difference so have to use Gamma Distribution function
    # not sure that this is correct
    # also abs_determinant() does not exist so I am only using log_abs_determinant
    return (tf.math.sqrt(np.pi)*self.scale.log_abs_determinant()*tfd.gamma((1+(-1+d)*k)/(2*k))) / \
           (tf.math.sqrt(kappa)*tfd.gamma((1+num_dims)/(2*kappa)))

def _prob(self, value):
    return self._unnormalized_prob(value) - self._normalization()

**Log PDF**

```
log(pdf) = coupled_log(coupled_exp(y^(a/2))^(-1/a) / Z)
log(pdf) = coupled_log(coupled_exp(y^(a/2))^(-1/a) - coupled_log(Z)
log(pdf) = y^(a/2)^(-1/a) - coupled_log(Z)
log(pdf) = y^((a/2)*(-1/a)) - coupled_log(Z)
log(pdf) = y^((a/2)*(-1/a)) - coupled_log(Z)
log(pdf) = y^(-2) - coupled_log(Z)

_log_unnormalized_prob = y^(-2)
_log_unnormalized_prob = (inv(Sigma)*(x - loc))^(-2)

_log_normalization = coupled_log(Z)
_log_normalization = coupled_log([sqrt(pi) * abs(det(Sigma)) * Gamma((1+(-1+d)*k)/(2*k))] /                                                    [sqrt(k) * Gamma((1+dk)/(2k)]
                                )
_log_normalization = coupled_log(sqrt(pi)) + coupled_log(abs(det(Sigma))) + 
                     coupled_log(Gamma((1+(-1+d)*k)/(2*k))) - coupled_log(sqrt(ki)) -
                     coupled_log(Gamma((1+dk)/(2k)))
```

Therefore:

_log_prob(x; mu, sigma) = _log_unnormalized_prob - _log_normalization

In [6]:
# HAVE NOT TESTED YET

def _log_unnormalized_prob(self, value):
    kappa = tf.convert_to_tensor(self.kappa)
    value = value - self._loc
    value = self.scale.solve(value[..., tf.newaxis])
    # mahalanobis is equivalent to y
    mahalanobis = tf.norm(value, axis=[-1, -2])
    # Use coupled_log instead of log? If so, does it cancel out coupled_exp?
    return mahalanobis^(-2)
    
def _log_normalization(self):
    kappa = tf.convert_to_tensor(self.kappa)
    num_dims = tf.cast(self.event_shape_tensor()[0], self.dtype)
    # may need to write a coupled function for log_abs_determinant() and log_gamma_difference()
    return coupled_log(tf.math.rsqrt(np.pi), kappa, num_dims) + \
           coupled_log(self.scale, kappa, num_dims) + \
           coupled_log(tfd.gamma((1+(-1+num_dims)*kappa)/(2*kappa)), kappa, num_dims) + \
           coupled_log(tf.math.rsqrt(kappa), kappa, num_dims) + \
           coupled_log(tfd.gamma((1+num_dims)/(2*kappa)), kappa, num_dims)

    return (tfp_math.log_gamma_difference(num_dims / 2., df / 2.) +
            num_dims / 2. * (tf.math.log(df) + np.log(np.pi)) +
            self.scale.log_abs_determinant())

def _log_prob(self, value):
    return self._log_unnormalized_prob(value) - self._log_normalization()